# SQL

## Table of Contents
- Conditionals (CASE)
- Subqueries
- Correlated Queries
- Nested Queries
- Common Table Expressions
- Window Functions

## Setup

In [1]:
import numpy as np
import pandas as pd
import sqlite3
import kaggle

from IPython.display import display

pd.options.display.max_rows = 200

In [2]:
kaggle.api.authenticate()

In [3]:
path = "../../../data/sql_data/"

In [4]:
kaggle.api.dataset_download_files(
    'hugomathien/soccer', 
    path=path, 
    unzip=True
)

In [5]:
ls ../../../data/sql_data

database.sqlite


In [6]:
database = path + 'database.sqlite'

In [7]:
def create_connection(db_file):
    """ create a database connection to the SQLite database
        specified by db_file
    """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        return conn
    except Error as e:
        print(e)
    return conn

In [8]:
conn = create_connection(database)

In [24]:
def make_metadata_df(con):
    cur = conn.cursor()
    tablename_statement = "SELECT * FROM sqlite_master WHERE type='table';"
    cur.execute(tablename_statement)
    meta_cols = {row[0]: num for num, row in enumerate(cur.description)}
    res = []
    print("** Tables in DB: \n")
    for row in cur:
        table_name = row[meta_cols["name"]]
        print(table_name)
        cols_statement = f"SELECT * from {table_name} LIMIT 1"
        new_cur = conn.cursor()
        new_cur.execute(cols_statement)
        for row in new_cur.description:
            res.append(dict(table_name=table_name,
                            col=row[0]))
        new_cur.close()
    cur.close()
    df = pd.DataFrame(res)
    return df

In [25]:
meta_df = make_metadata_df(conn)

** Tables in DB: 

sqlite_sequence
Player_Attributes
Player
Match
League
Country
Team
Team_Attributes


In [26]:
meta_df.head()

,table_name,col
0,sqlite_sequence,name
1,sqlite_sequence,seq
2,Player_Attributes,id
3,Player_Attributes,player_fifa_api_id
4,Player_Attributes,player_api_id


In [12]:
meta_df.table_name.unique()

array(['sqlite_sequence', 'Player_Attributes', 'Player', 'Match',
       'League', 'Country', 'Team', 'Team_Attributes'], dtype=object)

In [27]:
meta_df.table_name.value_counts()

Match                115
Player_Attributes     42
Team_Attributes       25
Player                 7
Team                   5
League                 3
sqlite_sequence        2
Country                2
Name: table_name, dtype: int64

In [11]:
meta_df.query("table_name == 'Match'")["col"].unique()

array(['id', 'country_id', 'league_id', 'season', 'stage', 'date',
       'match_api_id', 'home_team_api_id', 'away_team_api_id',
       'home_team_goal', 'away_team_goal', 'home_player_X1',
       'home_player_X2', 'home_player_X3', 'home_player_X4',
       'home_player_X5', 'home_player_X6', 'home_player_X7',
       'home_player_X8', 'home_player_X9', 'home_player_X10',
       'home_player_X11', 'away_player_X1', 'away_player_X2',
       'away_player_X3', 'away_player_X4', 'away_player_X5',
       'away_player_X6', 'away_player_X7', 'away_player_X8',
       'away_player_X9', 'away_player_X10', 'away_player_X11',
       'home_player_Y1', 'home_player_Y2', 'home_player_Y3',
       'home_player_Y4', 'home_player_Y5', 'home_player_Y6',
       'home_player_Y7', 'home_player_Y8', 'home_player_Y9',
       'home_player_Y10', 'home_player_Y11', 'away_player_Y1',
       'away_player_Y2', 'away_player_Y3', 'away_player_Y4',
       'away_player_Y5', 'away_player_Y6', 'away_player_Y7',
       'aw

In [13]:
def peek_sql_res(statement, con=conn):
    res_df = pd.read_sql(statement, con=conn)
    print(f"shape: {res_df.shape}")
    display(res_df.head(10))

## CASE

- When you have multiple logical conditions in a CASE statement, you may quickly end up with a large number of WHEN clauses to logically test every outcome you are interested in. 
- It's important to make sure you don't accidentally exclude key information in your ELSE clause.

#### Retrieve information about matches played between Barcelona (id = 8634) and Real Madrid (id = 8633)

In [14]:
statement = \
"""
SELECT  
    m.date,
    t.team_long_name AS opponent,
    CASE WHEN m.home_team_goal < m.away_team_goal THEN 'Barcelona win'
        WHEN m.home_team_goal > m.away_team_goal THEN 'Barcelona loss' 
        ELSE 'Tie' END AS outcome
FROM Match AS m
-- Join team to match
LEFT JOIN Team AS t 
ON m.home_team_api_id = t.team_api_id
WHERE m.away_team_api_id = 8634;
"""

In [15]:
# pd.read_sql("select * from Match limit 2", con=conn).T

In [16]:
peek_sql_res(statement)

shape: (152, 3)


,date,opponent,outcome
0,2008-08-31 00:00:00,CD Numancia,Barcelona loss
1,2008-11-16 00:00:00,RC Recreativo,Barcelona win
2,2008-11-29 00:00:00,Sevilla FC,Barcelona win
3,2008-12-21 00:00:00,Villarreal CF,Barcelona win
4,2009-01-11 00:00:00,CA Osasuna,Barcelona win
5,2009-02-01 00:00:00,Racing Santander,Barcelona win
6,2009-02-14 00:00:00,Real Betis Balompié,Tie
7,2009-03-01 00:00:00,Atlético Madrid,Barcelona loss
8,2009-03-15 00:00:00,UD Almería,Barcelona win
9,2009-04-04 00:00:00,Real Valladolid,Barcelona win


In [17]:
statement = \
"""
SELECT
    date,
    CASE WHEN m.home_team_api_id = 8634 THEN 'Barcelona' ELSE 'Madrid' END AS HOME,
    CASE WHEN m.away_team_api_id = 8634 THEN 'Barcelona' ELSE 'Madrid' END AS AWAY,
    CASE WHEN m.home_team_goal > m.away_team_goal THEN 'Home team win'
         WHEN m.away_team_goal > m.home_team_goal THEN 'Away team win'
         ELSE 'tie'
         END
    AS OUTCOME
FROM Match as m
WHERE (m.home_team_api_id = 8634 AND m.away_team_api_id = 8633)
OR
(m.home_team_api_id = 8633 AND m.away_team_api_id = 8634)
"""

In [18]:
peek_sql_res(statement)

shape: (16, 4)


,date,HOME,AWAY,OUTCOME
0,2008-12-13 00:00:00,Barcelona,Madrid,Home team win
1,2009-05-02 00:00:00,Madrid,Barcelona,Away team win
2,2009-11-29 00:00:00,Barcelona,Madrid,Home team win
3,2010-04-10 00:00:00,Madrid,Barcelona,Away team win
4,2010-11-29 00:00:00,Barcelona,Madrid,Home team win
5,2011-04-16 00:00:00,Madrid,Barcelona,tie
6,2011-12-10 00:00:00,Madrid,Barcelona,Away team win
7,2012-04-21 00:00:00,Barcelona,Madrid,Away team win
8,2013-03-02 00:00:00,Madrid,Barcelona,Home team win
9,2012-10-07 00:00:00,Barcelona,Madrid,tie


In [19]:
statement = \
"""
SELECT
    date,
    CASE WHEN m.home_team_api_id = 8634 THEN 'Barcelona' ELSE 'Madrid' END AS HOME,
    CASE WHEN m.away_team_api_id = 8634 THEN 'Barcelona' ELSE 'Madrid' END AS AWAY,
    CASE WHEN m.home_team_api_id = 8634 AND m.home_team_goal > m.away_team_goal THEN 'Barcelona win'
         WHEN m.home_team_api_id = 8634 AND m.home_team_goal < m.away_team_goal THEN 'Madrid win'
         WHEN m.away_team_api_id = 8634 AND m.home_team_goal > m.away_team_goal THEN 'Madrid win'
         WHEN m.away_team_api_id = 8634 AND m.home_team_goal < m.away_team_goal THEN 'Barcelona win'
         ELSE 'tie'
         END
         AS OUTCOME
    
FROM Match as m
WHERE (m.home_team_api_id = 8634 AND m.away_team_api_id = 8633)
    OR
    (m.home_team_api_id = 8633 AND m.away_team_api_id = 8634)
"""

In [20]:
peek_sql_res(statement)

shape: (16, 4)


,date,HOME,AWAY,OUTCOME
0,2008-12-13 00:00:00,Barcelona,Madrid,Barcelona win
1,2009-05-02 00:00:00,Madrid,Barcelona,Barcelona win
2,2009-11-29 00:00:00,Barcelona,Madrid,Barcelona win
3,2010-04-10 00:00:00,Madrid,Barcelona,Barcelona win
4,2010-11-29 00:00:00,Barcelona,Madrid,Barcelona win
5,2011-04-16 00:00:00,Madrid,Barcelona,tie
6,2011-12-10 00:00:00,Madrid,Barcelona,Barcelona win
7,2012-04-21 00:00:00,Barcelona,Madrid,Madrid win
8,2013-03-02 00:00:00,Madrid,Barcelona,Madrid win
9,2012-10-07 00:00:00,Barcelona,Madrid,tie


### Using `CASE` within a `WHERE` clause

- Can use a CASE statement as a filter in the WHERE statement to remove output you don't want to see

In [82]:
meta_df.head()

,table_name,col
0,sqlite_sequence,name
1,sqlite_sequence,seq
2,Player_Attributes,id
3,Player_Attributes,player_fifa_api_id
4,Player_Attributes,player_api_id


In [60]:
meta_df.query("table_name == 'Team'")

,table_name,col
171,Team,id
172,Team,team_api_id
173,Team,team_fifa_api_id
174,Team,team_long_name
175,Team,team_short_name


In [85]:
statement = \
"""
SELECT team_api_id, team_short_name, team_long_name 
FROM Team 
WHERE team_long_name = 'Bologna'
"""

In [86]:
peek_sql_res(statement)

shape: (1, 3)


,team_api_id,team_short_name,team_long_name
0,9857,BOL,Bologna


In [99]:
statement = \
"""
-- Select the season, date, home_team_goal, and away_team_goal columns
SELECT 
    m.season,
    m.date,
    m.home_team_goal,
    m.away_team_goal,
    t.team_long_name as home_team_name,
    t2.team_long_name as away_team_name
FROM match as m
LEFT JOIN Team as t
ON t.team_api_id = m.home_team_api_id
LEFT JOIN Team as t2
ON t2.team_api_id = m.away_team_api_id
WHERE 
-- Exclude games not won by Bologna
    CASE WHEN home_team_api_id = 9857 AND home_team_goal > away_team_goal THEN 'Bologna Win'
        WHEN away_team_api_id = 9857 AND away_team_goal > home_team_goal THEN 'Bologna Win' 
        END IS NOT NULL;
"""

In [100]:
peek_sql_res(statement)

shape: (68, 6)


,season,date,home_team_goal,away_team_goal,home_team_name,away_team_name
0,2008/2009,2008-08-31 00:00:00,1,2,Milan,Bologna
1,2008/2009,2008-12-13 00:00:00,5,2,Bologna,Torino
2,2008/2009,2009-01-18 00:00:00,1,2,Catania,Bologna
3,2008/2009,2009-01-28 00:00:00,0,1,Atalanta,Bologna
4,2008/2009,2009-03-08 00:00:00,3,0,Bologna,Sampdoria
5,2008/2009,2009-04-26 00:00:00,2,0,Bologna,Genoa
6,2008/2009,2009-05-17 00:00:00,2,1,Bologna,Lecce
7,2008/2009,2009-05-31 00:00:00,3,1,Bologna,Catania
8,2008/2009,2008-10-19 00:00:00,3,1,Bologna,Lazio
9,2009/2010,2009-10-28 00:00:00,2,1,Bologna,Siena


#### Calculating percent with CASE and AVG
CASE statements will return any value you specify in your THEN clause. 

This is an incredibly powerful tool for robust calculations and data manipulation when used in conjunction with an aggregate statement. 

One key task you can perform is using CASE inside an AVG function to calculate a percentage of information in your database.

In [49]:
meta_df.query("table_name == 'Country'")

,table_name,col
169,Country,id
170,Country,name


In [46]:
statement = \
"""
SELECT 
    c.name AS country,
    -- Round the percentage of tied games to 2 decimal points
    ROUND(AVG(CASE 
             WHEN m.season='2013/2014' AND m.home_team_goal = m.away_team_goal THEN 1
             WHEN m.season='2013/2014' AND m.home_team_goal != m.away_team_goal THEN 0
             END),2) AS pct_ties_2013_2014,
    ROUND(AVG(CASE 
             WHEN m.season='2014/2015' AND m.home_team_goal = m.away_team_goal THEN 1
             WHEN m.season='2014/2015' AND m.home_team_goal != m.away_team_goal THEN 0
             END),2) AS pct_ties_2014_2015
FROM Country AS c
LEFT JOIN match AS m
ON c.id = m.country_id
GROUP BY country;
"""

In [47]:
peek_sql_res(statement)

shape: (11, 3)


,country,pct_ties_2013_2014,pct_ties_2014_2015
0,Belgium,0.17,0.25
1,England,0.21,0.24
2,France,0.28,0.23
3,Germany,0.21,0.27
4,Italy,0.24,0.32
5,Netherlands,0.27,0.24
6,Poland,0.30,0.28
7,Portugal,0.25,0.28
8,Scotland,0.22,0.19
9,Spain,0.23,0.24
